In [1]:
# !pip install --upgrade pip

In [2]:
# !pip install python-pptx
# !pip install transformers
# !pip3 install torch torchvision torchaudio
# !conda install -c pytorch pytorch
# !conda install -c conda-forge tensorflow keras

In [18]:
# https://huggingface.co/nlpconnect/vit-gpt2-image-captioning

In [11]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image
from tqdm import tqdm

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image_paths):
  images = []
  for image_path in tqdm(image_paths):
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

/opt/anaconda3/envs/ppt_3_7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/ppt_3_7/lib/python3.7/site-packages/transformers/models/vit/feature_extraction_vit.py:31: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  FutureWarning,


In [12]:
predict_step(list(ppt.img_dict.values()))

100%|██████████| 38/38 [00:00<00:00, 136.76it/s]


['a blue and white photo of a person holding a remote control',
 'a close up picture of a clock on a wall',
 'an aerial view of a piece of artwork on a piece of paper',
 'a painting of a person holding a toy elephant',
 'a series of photographs showing a clock on a wall',
 'an aerial view of a clock on a wall',
 'a collage of photos showing different types of electronic devices',
 'a series of images showing a person holding a blue and white disk',
 'a collage of photos showing different types of electronic devices',
 'a collage of photos of a person holding an electronic device',
 'a black and white photo of a clock on a wall',
 'a small kitten sitting on the floor in front of a mirror',
 'a white dog is looking at the camera',
 'a gray and white cat sitting on top of a sandy surface',
 'a brown and white dog laying on the floor',
 'a cat sitting on the floor looking at the camera',
 'a wooden table topped with lots of different colored balls',
 'a blue and white photo of a person on 

In [3]:
# pip install Pillow
# pip install pdfminer.six
# pip install PyMuPDF

In [22]:
import fitz

# Open the PDF file
pdf_document = fitz.open("data/original/test.pdf")

# Iterate through each page of the PDF
for page_number in range(pdf_document.page_count):
    page = pdf_document[page_number]
    
    # Get the page's size
    page_width = page.bound().width
    page_height = page.bound().height
    
    # Create a pixmap with the same size as the page
    pixmap = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)
    
    # Save the pixmap as a PNG file
    png_filename = f"page_{page_number + 1}.png"
    pixmap.save(png_filename)
    
    # Free up memory
    pixmap = None

# Close the PDF file
pdf_document.close()


In [42]:
import fitz

file_path = "data/original/test3.pdf"
open_file = fitz.open(file_path)


name = 'test3'
img_num = 0
for page_number in range(len(open_file)):
    page = open_file[page_number]
    list_image = page.get_images()

    if list_image:
        print(f"{len(list_image)} images found on page {page_number}")
        
        for image in list_image:
            # Get the pixmap of the image
            pixmap = fitz.Pixmap(open_file, image[0])
            if pixmap.height >= 200 and pixmap.width >= 200:
                # self.img_num  - img_num
                img_num += 1
                num = str(img_num).zfill(3)
                image_filename = f"data/image/{name}/image_{num}.jpeg"
                pixmap.save(image_filename)
    else:
        print("No images found on page", page_number)

open_file.close()

2 images found on page 0
No images found on page 1
2 images found on page 2
2 images found on page 3
2 images found on page 4
No images found on page 5
1 images found on page 6
No images found on page 7
5 images found on page 8
2 images found on page 9


In [26]:
from pdfminer.high_level import extract_text

text = extract_text("data/original/test.pdf")
print(text)

2021 통계데이터 분석활용대회
서울특별시
‘여성 1인가구 안심홈세트’ 정책
분배적절성 분석

Index

1. 서론

분석배경 및 목적

2. 본론 – R 분석

데이터 선정

분석 코딩북

데이터 전처리

데이터 분석

각 변수의 분포 시각화

3. 본론 – Python 분석 Kmeans 군집화를 통한 군집유형 분석

분석결과 및 해석

정책분석 및 평가

4. 결론 - 분석활용전략 기대효과

방향제시

1. 서론

분석배경 및 목적

1인가구의 증가로 인한 연구 필요성 대두

서울시의 1인가구 수는 2000년 50만명에서부터 2015년 100만명을 넘기고, 2019년에는 약 130만명에 이를만큼 꾸준히
증가하고 있다. 또한 주된 가구 구성방식이 핵가족 단위에서 1인가구 단위로 바뀌어가고 있다. 학계에서도 많은 연구가
이루어지고 있기에 지자체에서도 이에 주목하고, 구체적 정책을 제시해야 한다.
서울특별시 1인가구 대책정책 연구(서울시, 2014)에서는 1인가구 정책을 ’주거주택 부문’, ‘사회적안전망 부문’, ‘공유사회
부문’ 세 가지로 나누고 있는데, 본 연구에서는 ‘사회적안전망 부문’에 주목해 보았다.

여성 1인가구에 대한 정책지원 미비

1인가구 밀집지역은 비밀집지역에 비해 2~3배 높은 범죄발생율을 보인다. 또한 주요 5대 강력범죄가 주거지에서 가장 많이
발생하며, 여성 피해자의 수도 꾸준히 증가세를 보이고 있다.
빅카인즈를 통해 2019년 5월 ‘신림동 강간미수 사건’ 전후로 ’1인가구 and 여성’ 키워드로 연관어 분석을 해보았을 때,
주거공간에서의 범죄불안 및 지원정책에 대한 수요가 증가했음을 알 수 있었다. 따라서 본 연구에서는 정책지원 필요성이
크게 증가했음을 인지하고, 이에 대해 구체적인 지원이 이루어져야 하기에 여성 1인가구에 대한 정책을 찾아 분석했다.

서울시 안심홈세트 지원 서비스 정책 분석

2019년부터 전국 지자체에서 여성 1인가구를 대상으로 한 안심지원사업을 시범적으로 운영하기 시작했고, 2021년
6월부터 서울시에서는

In [2]:
# pip install pdfminer.six

In [30]:
# from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# from pdfminer.converter import TextConverter
# from pdfminer.layout import LAParams
# from pdfminer.pdfpage import PDFPage
# from io import StringIO

# def get_pdf_text(filename):
#     rsrcmgr = PDFResourceManager()
#     retstr = StringIO()
#     codec = "utf-8"
#     laparams = LAParams()
#     device = TextConverter(
#         rsrcmgr, 
#         retstr, 
#         codec=codec, 
#         laparams=laparams
#     )
#     fp = open(filename, 'rb')
#     interpreter = PDFPageInterpreter(rsrcmgr, device)
#     password = ""
#     maxpages = 0
#     caching = True
#     pagenos = set()

#     pages = PDFPage.get_pages(
#         fp, 
#         pagenos, 
#         maxpages=maxpages, 
#         password=password, 
#         caching=caching, 
#         check_extractable=True
#     )
#     for page in pages:
#         interpreter.process_page(page)

#     text = retstr.getvalue()
#     fp.close()
#     device.close()
#     retstr.close()
#     return text

In [31]:
# get_pdf_text('data/original/test2.pdf')

In [35]:
import PyPDF2

# Open the PDF file in binary mode
with open('data/original/test2.pdf', 'rb') as f:

    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(f)

    # Get the total number of pages
    num_pages = pdf_reader.pages

    # Loop through each page and extract the text
    text = ''
    for page in num_pages:
        text += page.extract_text()

# Print the extracted text
# print(text)

PdfReadError("Invalid Elementary Object starting with b'B' @2891061: b'oPubWorld\\xb9\\xd9\\xc5\\xc1\\xc3\\xbc_Pro Bold\\n/Encoding /Identity-H\\n\\n/DescendantFonts [ 1121 0 R\\n ] \\n'")
PdfReadError("Invalid Elementary Object starting with b'B' @2890814: b'oPubWorld\\xb9\\xd9\\xc5\\xc1\\xc3\\xbc_Pro Bold\\n/Encoding /Identity-H\\n\\n/DescendantFonts [ 1115 0 R\\n ] \\n'")


In [36]:
# from tika import parser

# pdf_test = "test.pdf"
# pdf_parsed = parser.from_file(pdf_test)
# txt = open('icnairport.txt', 'w', encoding = 'utf-8')
# print(pdf_parsed['content'], file = txt)
# txt.close()

In [2]:
text2 = ''.join(text.split('\n'))

In [3]:
# pip install kiwipiepy

In [7]:
from hanspell import spell_checker

spelled_sent = spell_checker.check(text2)
checked_sent = spelled_sent.checked
print(checked_sent)

In [8]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

kiwi.space(text2)

'서울 특별시‘여성 1 인 가구 안심 홈 세트’정책 분배 적절성 분석 2021 통계 데이터 분석 활용 대회 Index1. 서론 분석 배경 및 목적 2. 본론 –R 분석 데이터 선정 분석 코딩 북 데이터 전처리 데이터 분석 각 변수의 분포 시각화 3. 본론 –Python 분석 Kmeans 군집화를 통한 군집 유형 분석 분석 결과 및 해석 정책 분석 및 평가4. 결론-분석 활용 전략 기대 효과 방향제 시 1. 서론 분석 배경 및 목적 서울시 안심 홈 세트 지원 서비스 정책 분석 2019년부터 전국 지자체에서 여성 1 인 가구를 대상으로 한 안심 지원 사업을 시범적으로 운영하기 시작했고,2021년 6월부터 서울시에서는 사업을 더 많은 자치구로 확대했다. 정 책에 대한 만족도는 높았으나, 여성 1 인 가구 수에 비해 지원은 턱없이 부족했다. 이는 시범 운영이 이루어지던 시기였기에 차차 지원 범위를 확대해 나가야한다. 따라서 본 연구자들은 서울시 안심 홈 세트 지원 정책을 활성화하기 위해 R,Python 등을 이용해 서울시의 행정 구별 여러 변수를 분석하고, 시각화하여 지자체에서‘우선 지원 분배 행정 구 선택 방법’을 제안하고자한다. 여성 1 인 가구에 대한 정책 지원 미비 1 인 가구 밀집 지역은 비 밀집 지역에 비해 2~3 배 높은 범죄 발생율을 보인다. 또한 주요 5대 강력 범죄가 주거지에서 가장 많이 발생하며, 여성 피해자의 수도 꾸준히 증가세를 보이고 있다. 빅 카인즈를 통해 2019년 5월‘신림동 강간 미수 사건’전 후로’1 인 가구 and 여성’키워드로 연관 어 분석을 해 보았을 때, 주거 공간에서의 범죄 불안 및 지원 정책에 대한 수요가 증가했음을 알 수 있었다. 따라서 본 연구에서는 정책 지원 필요성이 크게 증가했음을 인지하고, 이에 대해 구체적인 지원이 이루어져야하 기에 여성 1 인 가구에 대한 정책을 찾아 분석했다.1 인 가구의 증가로 인한 연구 필요성 대두 서울시의 1 인 가구 수는 2000년 50 만 명에서부터 2015년 100 만 명을 넘기고,

In [2]:
# !pip install pdf2docx

In [16]:
# !pip install docx2txt

In [19]:
import pdf2docx
import docx2txt

# Define input and output file paths
pdf_file = 'test2.pdf'
docx_file = 'test.docx'

# Convert PDF to DOCX
pdf2docx.parse(pdf_file, docx_file)

# Extract text from DOCX
text = docx2txt.process(docx_file)

# Print extracted text
print(text)

[INFO] Start to convert test2.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[WARNING] Ignore image due to inconsistent size of color and mask pixmaps: [255, 254, 255, 73, 8, 'DeviceRGB', '', 'Im157', 'DCTDecode', 0]
[WARNING] Ignore image due to inconsistent size of color and mask pixmaps: [257, 256, 126, 71, 8, 'DeviceRGB', '', 'Im159', 'DCTDecode', 0]
[WARNING] Ignore image due to inconsistent size of color and mask pixmaps: [380, 379, 272, 93, 8, 'DeviceRGB', '', 'Im175', 'DCTDecode', 0]
[WARNING] Ignore Line "최적 위치 선정" due to overlap
[WARNING] Ignore Line "시범 운영" due to overlap
[WARNING] Ignore Line "<image>" due to overlap
[WARNING] Ignore Line "<image>" due to overlap
[WARNING] Ignore Line "<image>" due to overlap
[WARNING] Ignore Line "<image>" due to overlap
[WARNING] Ignore Line "<image>" due to overlap
[WARNING] Ignore Line "<image>" due to overlap
[WARNING] Ignore Line "<image>" due to overlap
[WARNING] Ignore Line "<image>" due to overlap
[WARNING]

「2021년 통계데이터 분석·활용대회」























































































목차

A table of contents



1

분석 배경





o 
o 
o

주제 선정 
분석 목적 
분석 전략



2

데이터분석





o 
o

자료 출처 및 지표 세부 내역 데이터 분석



3

분석 결과





o

Elbow method



o

K-means Clustering



4

분석 활용 전략





o 
o

기대 효과 
실현 가능성













「2021년 통계데이터 분석·활용대회」















Part 1 분석 배경



주제 선정 | 분석 목적 | 분석 전략























「2021년 통계데이터 분석·활용대회」

1

분석배경



o

주제선정



•

문제 현황

탄소배출절감을위한관심노력

정부

일회용컵사용량

일회용컵재활용률

커피전문점수

서울시





출처: 김한상, 「11년 새 21억개 증가' 일회용컵







사용량·회수율 2022년 6월 보증금제 시행」,







『아주경제』, 2020.06.03.



탄소배출

심각한환경오염















「2021년 통계데이터 분석·활용대회」

1

분석배경











o

주제선정











•

분석 필요성











기존 일회용컵 보증금제





2022 일회용컵 보증금제





텀블링





•

2002~2008년 시행

•

2022년부터 재시행 예정

개선









•









•

일회용컵 이용시



‘자원순환보증금관리위원회’ 설치







보증금이 포함된 가격으로 구매 후











매장에 컵을 반환하면 보증금을

•

‘자원재활용법’을 따름(자발적 협약X)







돌려받는 제도













•